# Electricity Generation from api.electricitymap.org
---

SECTION 1 
collect data on carbon intensity, zones within SW , and power breakdown from the electricitymap api

In this file, I'm trying to update the collected data to include the datetime data for when the data were updated.  This is in the hope that I will be able to use this information to eliminate any rows that are duplicate times, deleting the older row.

The working file is the Elec_Generation_USjsonorig.ipynb

In [5]:
# Dependencies and Setup

import pandas as pd
import requests
import json


In [3]:

# zones for electrical utilities in US
zones = ["US-SW-PNM", "US-SW-EPE", "US-SW-WALC", "US-NW-PACE", "US-NW-PSCO", "US-CENT-SWPP", "US-TEX-ERCO", "US-MIDW-AECI","US-SW-AZPS","US-SW-AZPS",
         "US-NW-WACM", "US-SW-SRP", "US-SW-TEPC", "US-CENT-SPA", "US-CAL-IID", "US-CAL-CISO", "US-CAL-BANC","US-CAL-BANC", "US-CAL-TIDC", 
          "US-CAR-CPLE", "US-CAR-CPLW", "US-CAR-DUK", "US-CAR-SC", "US-CAR-SCEG", "US-CAR-YAD", "US-FLA-FMPP", "US-FLA-FPC" , "US-FLA-FPL",
          "US-FLA-GVL" , "US-FLA-HST", "US-FLA-JEA", "US-FLA-SEC", "US-FLA-TAL", "US-FLA-TEC", "US-MIDW-AECI" , "US-MIDW-LGEE", "US-MIDW-MISO",
          "US-NE-ISNE", "US-NW-BPAT", "US-NW-CHPD", "US-NW-DOPD", "US-NW-GCPD", "US-NW-GRID",  "US-NW-IPCO" , "US-NW-NWMT", "US-NW-NEVP", 
           "US-NW-PACW",  "US-NW-PGE", "US-NW-PSEI", "US-NW-SCL", "US-NW-TPWR", "US-NW-WAUW", "US-NY-NYIS", "US-SE-SEPA", "US-SE-SOCO" , 
           "US-TEN-TVA"]


In [7]:

# get carbon intensity history for the US utilities
urls = []
for index, url in enumerate(zones):
    url = f'https://api.electricitymap.org/v3/carbon-intensity/history?zone={zones[index]}'
    urls.append(url)

responses_dict = {}
for idx, url in enumerate(urls):
    response = requests.get(url)
    responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file
import json

file_path = "C_intensity_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(responses_dict, json_file, indent=4)

print("Dictionary successfully exported to JSON file.")

df_carbon_intensity_history = pd.read_json(file_path)


Dictionary successfully exported to JSON file.


In [8]:
#request power breakdown
pburls = []
for index, url in enumerate(zones):
    pburl = f'https://api.electricitymap.org/v3/power-breakdown/history?zone={zones[index]}'
    pburls.append(pburl)

power_breakdown_responses_dict = {}
for idx, pburl in enumerate(pburls):
    response = requests.get(pburl)
    power_breakdown_responses_dict[f"response_{idx+1}"] = response.json()

# Specify the file path where you want to save the JSON file

file_path = "power_breakdown_history_data.json"

# Write the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(power_breakdown_responses_dict, json_file, indent=4)

df_power_breakdown_history = pd.read_json(file_path)

SECTION 2     
PowerBreakdown data transformation

In [9]:
# pull data from power breakdown json in dataframe
region = df_power_breakdown_history['response_1']['history'][0]['zone']
datetime = df_power_breakdown_history['response_1']['history'][0]['datetime']
update_time = df_power_breakdown_history['response_1']['history'][0]['updatedAt']
nuclear = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['nuclear']
geothermal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['geothermal']
biomass = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['biomass']
coal = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['coal']
wind = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['wind']
solar = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['solar']
hydro = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro']
gas = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['gas']
oil = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['oil']
unknown = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['unknown']
hydro_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['hydro discharge']
battery_discharge = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionBreakdown"]['battery discharge']
renewable_percentage = df_power_breakdown_history['response_1']['history'][0]["renewablePercentage"]
total_consumption = df_power_breakdown_history['response_1']['history'][0]["powerConsumptionTotal"]
estimated = df_power_breakdown_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1 = {'region':region,'datetime':datetime, 'update time': update_time, 'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}

# Create a dataFrame with the first values
df_US = pd.DataFrame.from_dict(us_pnm1,orient='index')


In [10]:
df_US

,0
region,US-SW-PNM
datetime,2024-07-23T15:00:00.000Z
update time,2024-07-23T23:49:27.580Z
nuclear,0
geothermal,0
biomass,0
coal,295
wind,0
solar,986
hydro,18


In [11]:
# Data wrangling from the response to create a legible dataFrame
# outer for loop for regions/responses
for reg in range(len(zones)):
    #for each zone
    response = f"response_{reg+1}"
   
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_power_breakdown_history[f"{response}"]['history'][i]['zone']
        datetime = df_power_breakdown_history[f"{response}"]['history'][i]['datetime']
        update_time = df_power_breakdown_history[f"{response}"]['history'][i]['updatedAt']
        nuclear = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['nuclear']
        geothermal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['geothermal']
        biomass = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['biomass']
        coal = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['coal']
        wind = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['wind']
        solar = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['solar']
        hydro = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro']
        gas = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['gas']
        oil = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['oil']
        unknown = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['unknown']
        hydro_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['hydro discharge']
        battery_discharge = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionBreakdown"]['battery discharge']
        renewable_percentage = df_power_breakdown_history[f"{response}"]['history'][i]["renewablePercentage"]
        total_consumption = df_power_breakdown_history[f"{response}"]['history'][i]["powerConsumptionTotal"]
        estimated = df_power_breakdown_history[f"{response}"]['history'][i]["isEstimated"]

        # this 24 is also for the 23 hours of data for each zone
        df_US[24*reg+i]= {'region':region, 'datetime':datetime, 'update time': update_time,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar, 
           'hydro':hydro, 'gas':gas, 'oil':oil, 'unknown':unknown, 'hydro-discharge':hydro_discharge, 
           'battery_discharge':battery_discharge, 'renewable_percentage':renewable_percentage, 'total_consumption':total_consumption, 
           'estimated':estimated}
#set up the times as rows and measurements as columns
df_US_new = df_US.transpose()

# check data types
df_US_new.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_33776\730698991.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_US[24*reg+i]= {'region':region, 'datetime':datetime, 'update time': update_time,'nuclear':nuclear,'geothermal':geothermal,'biomass':biomass, 'coal':coal, 'wind':wind, 'solar':solar,


,region,datetime,update time,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated
count,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1344,1333,1344,1344
unique,53,24,58,294,25,56,573,333,392,556,928,37,275,1,27,84,1146,2
top,US-CAL-BANC,2024-07-23T15:00:00.000Z,2024-07-23T23:49:27.580Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True
freq,48,56,166,820,1320,1201,610,779,708,333,192,1208,564,1344,1309,198,11,1221


In [12]:
# fill NA values with zeroes for energy values
df_US_new = df_US_new.fillna({'nuclear': 0,'geothermal': 0,'biomass': 0, 'coal': 0, 'wind': 0, 'solar': 0, 
           'hydro': 0, 'gas': 0, 'oil': 0, 'unknown': 0, 'hydro-discharge':0, 'renewable_percentage':0,
           'battery_discharge':0})
df_US_new.describe()

,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.0,1344.000000,1344.000000
mean,1146.568452,12.837054,18.579613,1626.728423,225.582589,480.718750,573.938988,4143.143601,1.491071,98.909970,0.0,19.243304,29.186012
std,2484.833572,95.415232,94.879025,4106.233086,850.504915,1916.664043,1219.617741,8280.581402,6.861131,235.300502,0.0,269.025039,33.776426
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,276.750000,0.000000,0.000000,0.0,0.000000,3.000000
50%,0.000000,0.000000,0.000000,45.000000,0.000000,0.000000,60.500000,1070.000000,0.000000,9.000000,0.0,0.000000,15.000000
75%,406.000000,0.000000,0.000000,1401.000000,83.250000,175.250000,633.750000,2563.000000,0.000000,75.250000,0.0,0.000000,45.000000
max,13646.000000,764.000000,593.000000,29705.000000,8617.000000,16958.000000,9292.000000,47892.000000,52.000000,2090.000000,0.0,5739.000000,100.000000


In [20]:
# convert measured Energy values to integers in Giga Watts
#convert_dict = {'hydro': int}
convert_dict = {'nuclear': int, 'geothermal': int, 'biomass': int, 'coal': int, 'wind': int, 'solar': int, 'hydro': int, 'gas': int, 'oil': int, 
              'hydro-discharge': int, 'battery_discharge': int, 'renewable_percentage': int, 'total_consumption': int
               }
 # note - the unknown column only has values rarely - converting null values to integer doesn't work so this is left as an object
df_US_new = df_US_new.astype(convert_dict)

#check that data types are changed to int
df_US_new.dtypes

region                          object
datetime                        object
update time                     object
nuclear                          int32
geothermal                       int32
biomass                          int32
coal                             int32
wind                             int32
solar                            int32
hydro                            int32
gas                              int32
oil                              int32
unknown                          int64
hydro-discharge                  int32
battery_discharge                int32
renewable_percentage             int32
total_consumption                int32
estimated                       object
UTC time                        object
UTC date                        object
UTC UpdateTime          datetime64[ns]
dtype: object

In [51]:
# Date Time work

# import datetime dependencies

from datetime import datetime

# set up lists to hold parsed data and DateTime as a datetime datetype
dates=[]
times = []
DateTime =[]
UpdateTime = []

# convert date time strings
for i in range(len(df_US_new['datetime'])):

    # Parse the timestamp string to a datetime object
    dt_obj = datetime.strptime(df_US_new.iloc[i,1], '%Y-%m-%dT%H:%M:%S.%fZ')
    dt_update_obj = datetime.strptime(df_US_new.iloc[i,2], '%Y-%m-%dT%H:%M:%S.%fZ')

    date = dt_obj.strftime('%Y-%m-%d')
    time = dt_obj.strftime('%H:%M:%S')

#add the new times and dates to lists

    dates.append(date)
    times.append(time)
    DateTime.append(dt_obj)
    UpdateTime.append(dt_update_obj)

# add the times and dates to new columns in the data frame
df_US_new['UTC time'] = times
df_US_new['UTC date'] = dates
df_US_new['UTC DateTime'] = DateTime
df_US_new['UTC UpdateTime'] = UpdateTime

In [52]:
#set the UTC DateTime as the index
df_US_new_reindex = df_US_new.set_index('UTC DateTime', inplace=True)

#drop the datetime column that contains a string
df_US_newer = df_US_new.drop(['datetime', 'update time'], axis=1)


df_US_newer.head()



,region,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,oil,unknown,hydro-discharge,battery_discharge,renewable_percentage,total_consumption,estimated,UTC time,UTC date,UTC UpdateTime
UTC DateTime,,,,,,,,,,,,,,,,,,,
2024-07-23 15:00:00,US-SW-PNM,0,0,0,295,0,986,18,350,0,0,0,0,61,1649,True,15:00:00,2024-07-23,2024-07-23 23:49:27.580
2024-07-23 16:00:00,US-SW-PNM,0,0,0,298,0,1011,18,368,0,0,0,0,61,1694,True,16:00:00,2024-07-23,2024-07-23 23:49:27.580
2024-07-23 17:00:00,US-SW-PNM,0,0,0,304,0,1023,18,395,0,0,0,0,60,1740,True,17:00:00,2024-07-23,2024-07-23 23:49:27.580
2024-07-23 18:00:00,US-SW-PNM,0,0,0,324,0,1023,18,425,0,0,0,0,58,1790,True,18:00:00,2024-07-23,2024-07-23 23:48:06.742
2024-07-23 19:00:00,US-SW-PNM,0,0,0,327,0,1005,18,443,0,0,0,0,57,1793,True,19:00:00,2024-07-23,2024-07-23 23:48:06.742


Section 3
Transform carbon intensity data

In [53]:
# pull data from C intensity json in dataframe
region = df_carbon_intensity_history['response_1']['history'][0]['zone']
datetime = df_carbon_intensity_history['response_1']['history'][0]['datetime']
intensity_update_time = df_carbon_intensity_history['response_1']['history'][0]['updatedAt']
carbon_Intensity = df_carbon_intensity_history['response_1']['history'][0]["carbonIntensity"]
estimated = df_carbon_intensity_history['response_1']['history'][0]["isEstimated"]

# create a dictionary with first values for this zone
us_pnm1C = {'region':region,'datetime':datetime, 'intensity_update_time': carbon_Intensity, 'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# Create a dataFrame with the first values
df_US_C = pd.DataFrame.from_dict(us_pnm1C,orient='index')

In [60]:
# Data wrangling from the response to create a legible dataFrame for carbon intensity history

# outer for loop for regions/responses
for reg in range(len(zones)):
    response = f"response_{reg+1}"
    
# pull data from json for each time in this file for this region and add to the existing dataframe
    for i in range(24):
        # 24 is for the 24 hours of data for each zone
        region = df_carbon_intensity_history[f"{response}"]['history'][i]['zone']
        datetime = df_carbon_intensity_history[f"{response}"]['history'][i]['datetime']
        intensity_update_time = df_carbon_intensity_history[f"{response}"]['history'][i]['updatedAt']
        carbon_Intensity = df_carbon_intensity_history['response_1']['history'][i]["carbonIntensity"]
        estimated = df_carbon_intensity_history['response_1']['history'][i]["isEstimated"]
        
        # this 24 is also for the 24 hours of data in each zone
        df_US_C[24*reg+i]= {'region':region, 'datetime':datetime, 'intensity_update_time': intensity_update_time, 'Carbon_Intensity':carbon_Intensity, 'estimated':estimated}

# make the datetime the rows and carbon_intensity a column
df_US_C_new = df_US_C.transpose()

# check data types
df_US_C_new.dtypes

region                   object
datetime                 object
intensity_update_time    object
Carbon_Intensity         object
estimated                object
dtype: object

In [61]:
# convert carbon intensity measurement to an integer in g CO2e/kWh
convert_dict_C= {'Carbon_Intensity': int}
 
df_US_C_new = df_US_C_new.astype(convert_dict_C)

#check that the datatype has been changed
df_US_C_new.dtypes

region                   object
datetime                 object
intensity_update_time    object
Carbon_Intensity          int32
estimated                object
dtype: object

In [62]:

# add the times and dates to new columns in the data frame   -    This assumes the data for carbon intensity is pulled at the same time as power breakdown
df_US_C_new['UTC time'] = times
df_US_C_new['UTC date'] = dates
df_US_C_new['UTC DateTime'] = DateTime

#set the UTC DateTime as the index
df_US_C_new_reindex = df_US_C_new.set_index('UTC DateTime', inplace=True)
#drop the datetime column that contains a string
df_US_C_newer = df_US_C_new.drop(['datetime'], axis=1)

In [63]:
df_US_C_newer.head()

,region,intensity_update_time,Carbon_Intensity,estimated,UTC time,UTC date
UTC DateTime,,,,,,
2024-07-23 15:00:00,US-SW-PNM,2024-07-23T23:49:27.580Z,353,True,15:00:00,2024-07-23
2024-07-23 16:00:00,US-SW-PNM,2024-07-23T23:49:27.580Z,351,True,16:00:00,2024-07-23
2024-07-23 17:00:00,US-SW-PNM,2024-07-23T23:49:27.580Z,355,True,17:00:00,2024-07-23
2024-07-23 18:00:00,US-SW-PNM,2024-07-23T23:48:06.742Z,368,True,18:00:00,2024-07-23
2024-07-23 19:00:00,US-SW-PNM,2024-07-23T23:48:06.742Z,375,True,19:00:00,2024-07-23


Section 4
Merge dataframes

In [64]:
df_power_and_carbon= pd.merge(df_US_newer, df_US_C_newer,on=['UTC DateTime','region','UTC time','UTC date'])



df_power_and_carbon.rename(columns={'Carbon_Intensity':'Carbon_Intensity(gCO2eq/kWh)','total_consumption':'total_consumption(GW)', 'nuclear':'nuclear(GW)', 
                                    'geothermal':'geothermal(GW)', 'biomass':'biomass(GW)', 'coal':'coal(GW)', 'wind':'wind(GW)', 'solar':'solar(GW)', 
                                    'hydro':'hydro(GW)','gas':'gas(GW)', 'region_x': 'region', 'estimated_x': 'breakdown estimated?','estimated_y':'intensity estimated?'}, inplace=True)




df_power_and_carbon.head()

,region,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,...,battery_discharge,renewable_percentage,total_consumption(GW),breakdown estimated?,UTC time,UTC date,UTC UpdateTime,intensity_update_time,Carbon_Intensity(gCO2eq/kWh),intensity estimated?
UTC DateTime,,,,,,,,,,,,,,,,,,,,,
2024-07-23 15:00:00,US-SW-PNM,0,0,0,295,0,986,18,350,0,...,0,61,1649,True,15:00:00,2024-07-23,2024-07-23 23:49:27.580,2024-07-23T23:49:27.580Z,353,True
2024-07-23 16:00:00,US-SW-PNM,0,0,0,298,0,1011,18,368,0,...,0,61,1694,True,16:00:00,2024-07-23,2024-07-23 23:49:27.580,2024-07-23T23:49:27.580Z,351,True
2024-07-23 17:00:00,US-SW-PNM,0,0,0,304,0,1023,18,395,0,...,0,60,1740,True,17:00:00,2024-07-23,2024-07-23 23:49:27.580,2024-07-23T23:49:27.580Z,355,True
2024-07-23 18:00:00,US-SW-PNM,0,0,0,324,0,1023,18,425,0,...,0,58,1790,True,18:00:00,2024-07-23,2024-07-23 23:48:06.742,2024-07-23T23:48:06.742Z,368,True
2024-07-23 19:00:00,US-SW-PNM,0,0,0,327,0,1005,18,443,0,...,0,57,1793,True,19:00:00,2024-07-23,2024-07-23 23:48:06.742,2024-07-23T23:48:06.742Z,375,True


In [17]:
# import previous cleaned file into a pandas dataframe
df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')
df_us_energy_reindex=df_us_energy.set_index("UTC DateTime")
df_us_energy_reindex.describe()

C:\Users\mrkol\AppData\Local\Temp\ipykernel_38088\2571967791.py:2: DtypeWarning: Columns (16,20,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_us_energy = pd.read_csv('data/runningUSenergy_data.csv')


,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18487.000000,18487.000000,...,48.0,48.0,48.0,48.000000,48.000000,48.000000,48.0,48.000000,48.000000,48.000000
mean,1220.980476,14.538424,22.651500,1702.290634,477.441835,507.401811,632.134769,4191.959976,5.186672,94.802348,...,0.0,0.0,0.0,351.833333,196.125000,423.166667,18.0,533.125000,1552.104167,358.500000
std,2557.723036,102.744695,106.961656,4456.546051,1819.233193,2040.485444,1432.700376,8238.658655,45.099045,202.405871,...,0.0,0.0,0.0,88.284437,336.617507,443.893668,0.0,127.405973,485.022888,148.050308
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,157.000000,0.000000,0.000000,18.0,310.000000,904.000000,212.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,232.000000,0.000000,0.000000,...,0.0,0.0,0.0,304.000000,0.000000,0.000000,18.0,422.250000,1191.500000,212.000000
50%,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,73.000000,962.000000,0.000000,8.000000,...,0.0,0.0,0.0,340.500000,21.500000,206.000000,18.0,546.500000,1534.500000,358.500000
75%,650.000000,0.000000,0.000000,1353.500000,130.000000,152.000000,672.000000,3319.000000,0.000000,90.000000,...,0.0,0.0,0.0,388.000000,239.750000,925.500000,18.0,643.000000,1761.250000,505.000000
max,13803.000000,812.000000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2081.000000,...,0.0,0.0,0.0,502.000000,1325.000000,1055.000000,18.0,718.000000,2701.000000,505.000000


In [18]:
df_us_energy_reindex.drop_duplicates(inplace=True)
df_us_energy_reindex.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18439.000000,18487.000000,18487.000000,...,48.0,48.0,48.0,48.000000,48.000000,48.000000,48.0,48.000000,48.000000,48.000000
mean,1220.980476,14.538424,22.651500,1702.290634,477.441835,507.401811,632.134769,4191.959976,5.186672,94.802348,...,0.0,0.0,0.0,351.833333,196.125000,423.166667,18.0,533.125000,1552.104167,358.500000
std,2557.723036,102.744695,106.961656,4456.546051,1819.233193,2040.485444,1432.700376,8238.658655,45.099045,202.405871,...,0.0,0.0,0.0,88.284437,336.617507,443.893668,0.0,127.405973,485.022888,148.050308
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,157.000000,0.000000,0.000000,18.0,310.000000,904.000000,212.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,232.000000,0.000000,0.000000,...,0.0,0.0,0.0,304.000000,0.000000,0.000000,18.0,422.250000,1191.500000,212.000000
50%,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,73.000000,962.000000,0.000000,8.000000,...,0.0,0.0,0.0,340.500000,21.500000,206.000000,18.0,546.500000,1534.500000,358.500000
75%,650.000000,0.000000,0.000000,1353.500000,130.000000,152.000000,672.000000,3319.000000,0.000000,90.000000,...,0.0,0.0,0.0,388.000000,239.750000,925.500000,18.0,643.000000,1761.250000,505.000000
max,13803.000000,812.000000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2081.000000,...,0.0,0.0,0.0,502.000000,1325.000000,1055.000000,18.0,718.000000,2701.000000,505.000000


In [19]:
# concatentate current data with existing file
df_both = pd.concat([df_us_energy_reindex,df_power_and_carbon])

# drop duplicate rows
df_both.drop_duplicates(inplace=True)
df_both.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,19042.000000,19042.000000,19042.000000,19042.000000,19042.000000,19042.000000,19042.000000,19042.000000,19090.000000,19090.000000,...,48.0,48.0,48.0,48.000000,48.000000,48.000000,48.0,48.000000,48.000000,48.000000
mean,1223.079929,14.541120,22.613591,1696.656811,473.489970,496.094003,627.916028,4188.940815,5.063332,94.442378,...,0.0,0.0,0.0,351.833333,196.125000,423.166667,18.0,533.125000,1552.104167,358.500000
std,2560.784157,102.742094,106.797475,4440.282283,1802.129733,2014.995657,1422.968856,8226.894607,44.404268,202.011928,...,0.0,0.0,0.0,88.284437,336.617507,443.893668,0.0,127.405973,485.022888,148.050308
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,157.000000,0.000000,0.000000,18.0,310.000000,904.000000,212.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,232.000000,0.000000,0.000000,...,0.0,0.0,0.0,304.000000,0.000000,0.000000,18.0,422.250000,1191.500000,212.000000
50%,0.000000,0.000000,0.000000,22.000000,0.000000,0.000000,72.000000,960.000000,0.000000,8.000000,...,0.0,0.0,0.0,340.500000,21.500000,206.000000,18.0,546.500000,1534.500000,358.500000
75%,650.000000,0.000000,0.000000,1349.000000,131.000000,141.000000,666.000000,3325.500000,0.000000,90.000000,...,0.0,0.0,0.0,388.000000,239.750000,925.500000,18.0,643.000000,1761.250000,505.000000
max,13803.000000,812.000000,657.000000,49357.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2090.000000,...,0.0,0.0,0.0,502.000000,1325.000000,1055.000000,18.0,718.000000,2701.000000,505.000000


In [20]:
df_both.to_csv(f'data/runningUSenergy_data.csv')

In [21]:
df_both_cleaned = df_both.loc[df_both['breakdown estimated?']==False,:]
df_both_cleaned.describe()

,nuclear(GW),geothermal(GW),biomass(GW),coal(GW),wind(GW),solar(GW),hydro(GW),gas(GW),oil,unknown,...,nuclear,geothermal,biomass,coal,wind,solar,hydro,gas,total_consumption,carbon_Intensity
count,5361.000000,5361.000000,5361.000000,5361.000000,5361.000000,5361.000000,5361.000000,5361.000000,5361.000000,5361.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1440.358702,49.738295,75.038799,1958.865324,931.272151,729.902071,1110.066779,5687.082820,12.407200,113.276441,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2280.962673,185.391662,184.033939,4683.535752,2529.682432,2737.881443,1885.725939,8285.370172,75.451203,221.336738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,264.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.000000,0.000000,0.000000,25.000000,33.000000,1.000000,250.000000,1539.000000,0.000000,13.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2253.000000,0.000000,10.000000,1118.000000,426.000000,194.000000,1366.000000,9484.000000,0.000000,127.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,13287.000000,812.000000,657.000000,48997.000000,21642.000000,19768.000000,65867.000000,54713.000000,2054.000000,2053.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_both_cleaned.to_csv(f'data/runningUSenergy_data_filtered.csv')